In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import ensemble 
from lightgbm import LGBMClassifier
from sklearn.linear_model import SGDClassifier
import os
from sklearn import ensemble
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler


c:\users\user\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print('Preprocessing data')
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

test_id = [ ]
for doc in test_df.Id:
    test_id.append(doc)
    
y_train = train_df.Cover_Type
x_train = train_df.drop('Cover_Type', axis=1)
x_train = x_train.drop('Id',axis=1)
test_df = test_df.drop('Id',axis=1)

Preprocessing data


In [3]:

x_train = x_train.drop('Soil_Type7',axis=1)
x_train = x_train.drop('Soil_Type8',axis=1)
x_train = x_train.drop('Soil_Type15',axis=1)
x_train = x_train.drop('Soil_Type25',axis=1)
# x_train = x_train.drop('Hillshade_3pm',axis=1)
# x_train = x_train.drop('Hillshade_Noon',axis=1)
# x_train = x_train.drop('Hillshade_9am',axis=1)
# x_train = x_train.drop('Slope',axis=1)
# x_train = x_train.drop('Aspect',axis=1)

test_df = test_df.drop('Soil_Type7',axis=1)
test_df = test_df.drop('Soil_Type8',axis=1)
test_df = test_df.drop('Soil_Type15',axis=1)
test_df = test_df.drop('Soil_Type25',axis=1)
# test_df = test_df.drop('Hillshade_9am',axis=1)
# test_df = test_df.drop('Hillshade_Noon',axis=1)
# test_df = test_df.drop('Hillshade_3pm',axis=1)
# test_df = test_df.drop('Slope',axis=1)
# test_df = test_df.drop('Aspect',axis=1)


x_train['HF1'] = x_train['Horizontal_Distance_To_Hydrology']+x_train['Horizontal_Distance_To_Fire_Points']
x_train['HF2'] = abs(x_train['Horizontal_Distance_To_Hydrology']-x_train['Horizontal_Distance_To_Fire_Points'])
x_train['HR1'] = abs(x_train['Horizontal_Distance_To_Hydrology']+x_train['Horizontal_Distance_To_Roadways'])
x_train['HR2'] = abs(x_train['Horizontal_Distance_To_Hydrology']-x_train['Horizontal_Distance_To_Roadways'])
x_train['FR1'] = abs(x_train['Horizontal_Distance_To_Fire_Points']+x_train['Horizontal_Distance_To_Roadways'])
x_train['FR2'] = abs(x_train['Horizontal_Distance_To_Fire_Points']-x_train['Horizontal_Distance_To_Roadways'])
x_train['distance'] = np.sqrt(np.array(x_train['Horizontal_Distance_To_Hydrology']**2 + x_train["Vertical_Distance_To_Hydrology"]**2))
x_train['Mean_Amenities']=(x_train.Horizontal_Distance_To_Fire_Points + x_train.Horizontal_Distance_To_Hydrology + x_train.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
x_train['Mean_Fire_Hyd']=(x_train.Horizontal_Distance_To_Fire_Points + x_train.Horizontal_Distance_To_Hydrology) / 2 
x_train['mean_hillshade'] = (x_train['Hillshade_9am']  + x_train['Hillshade_Noon']  + x_train['Hillshade_3pm'] ) / 3
x_train["Vertical_Distance_To_Hydrology"] = abs(x_train['Vertical_Distance_To_Hydrology'])
x_train['Mean_HorizontalHydrology_HorizontalFire'] = (x_train['Horizontal_Distance_To_Hydrology']+x_train['Horizontal_Distance_To_Fire_Points'])/2
x_train['Mean_HorizontalHydrology_HorizontalRoadways'] = (x_train['Horizontal_Distance_To_Hydrology']+x_train['Horizontal_Distance_To_Roadways'])/2
x_train['Mean_HorizontalFire_Points_HorizontalRoadways'] = (x_train['Horizontal_Distance_To_Fire_Points']+x_train['Horizontal_Distance_To_Roadways'])/2

x_train['MeanNeg_Mean_HorizontalHydrology_HorizontalFire'] = (x_train['Horizontal_Distance_To_Hydrology']-x_train['Horizontal_Distance_To_Fire_Points'])/2
x_train['MeanNeg_HorizontalHydrology_HorizontalRoadways'] = (x_train['Horizontal_Distance_To_Hydrology']-x_train['Horizontal_Distance_To_Roadways'])/2
x_train['MeanNeg_HorizontalFire_Points_HorizontalRoadways'] = (x_train['Horizontal_Distance_To_Fire_Points']-x_train['Horizontal_Distance_To_Roadways'])/2


# x_train['angle'] = x_train['Slope'] * x_train['Elevation']

test_df['HF1'] = test_df['Horizontal_Distance_To_Hydrology']+test_df['Horizontal_Distance_To_Fire_Points']
test_df['HF2'] = abs(test_df['Horizontal_Distance_To_Hydrology']-test_df['Horizontal_Distance_To_Fire_Points'])
test_df['HR1'] = abs(test_df['Horizontal_Distance_To_Hydrology']+test_df['Horizontal_Distance_To_Roadways'])
test_df['HR2'] = abs(test_df['Horizontal_Distance_To_Hydrology']-test_df['Horizontal_Distance_To_Roadways'])
test_df['FR1'] = abs(test_df['Horizontal_Distance_To_Fire_Points']+test_df['Horizontal_Distance_To_Roadways'])
test_df['FR2'] = abs(test_df['Horizontal_Distance_To_Fire_Points']-test_df['Horizontal_Distance_To_Roadways'])
test_df['distance'] = np.sqrt(np.array(test_df['Horizontal_Distance_To_Hydrology']**2 + test_df["Vertical_Distance_To_Hydrology"]**2))
test_df['Mean_Amenities']=(test_df.Horizontal_Distance_To_Fire_Points + test_df.Horizontal_Distance_To_Hydrology + test_df.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
test_df['Mean_Fire_Hyd']=(test_df.Horizontal_Distance_To_Fire_Points + test_df.Horizontal_Distance_To_Hydrology) / 2
test_df['mean_hillshade'] = (test_df['Hillshade_9am']  + test_df['Hillshade_Noon']  + test_df['Hillshade_3pm'] ) / 3
test_df["Vertical_Distance_To_Hydrology"] = abs(test_df['Vertical_Distance_To_Hydrology'])
test_df['Mean_HorizontalHydrology_HorizontalFire'] = (test_df['Horizontal_Distance_To_Hydrology']+test_df['Horizontal_Distance_To_Fire_Points'])/2
test_df['Mean_HorizontalHydrology_HorizontalRoadways'] = (test_df['Horizontal_Distance_To_Hydrology']+test_df['Horizontal_Distance_To_Roadways'])/2
test_df['Mean_HorizontalFire_Points_HorizontalRoadways'] = (test_df['Horizontal_Distance_To_Fire_Points']+test_df['Horizontal_Distance_To_Roadways'])/2

test_df['MeanNeg_Mean_HorizontalHydrology_HorizontalFire'] = (test_df['Horizontal_Distance_To_Hydrology']-test_df['Horizontal_Distance_To_Fire_Points'])/2
test_df['MeanNeg_HorizontalHydrology_HorizontalRoadways'] = (test_df['Horizontal_Distance_To_Hydrology']-test_df['Horizontal_Distance_To_Roadways'])/2
test_df['MeanNeg_HorizontalFire_Points_HorizontalRoadways'] = (test_df['Horizontal_Distance_To_Fire_Points']-test_df['Horizontal_Distance_To_Roadways'])/2
# test_df['angle'] = test_df['Slope'] * test_df['Elevation']




In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565892 entries, 0 to 565891
Data columns (total 66 columns):
Elevation                                           565892 non-null int64
Aspect                                              565892 non-null int64
Slope                                               565892 non-null int64
Horizontal_Distance_To_Hydrology                    565892 non-null int64
Vertical_Distance_To_Hydrology                      565892 non-null int64
Horizontal_Distance_To_Roadways                     565892 non-null int64
Hillshade_9am                                       565892 non-null int64
Hillshade_Noon                                      565892 non-null int64
Hillshade_3pm                                       565892 non-null int64
Horizontal_Distance_To_Fire_Points                  565892 non-null int64
Wilderness_Area1                                    565892 non-null int64
Wilderness_Area2                                    565892 non-null int64
W